In [1]:
from PIL import Image
import numpy as np
import torch
from torch import nn
from torch import optim
from math import inf
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

In [2]:
a = Image.open("train/cat.0.jpg")
# cats: 0 -> 12499; dogs: 0 -> 12499

# resize to 128 x 128

"""
0 = Cat
1 = Dog
"""

X = np.zeros((25000, 128, 128, 3), dtype = np.float32)
labels = np.zeros(25000, dtype=np.float32)
k = 0
for i in range(12500):
    im = Image.open(f"train/cat.{i}.jpg")
    im = im.resize((128,128))
    im = np.array(np.array(im))
    labels[k] = 0
    X[k] = im
    k += 1
    im = Image.open(f"train/dog.{i}.jpg")
    im = im.resize((128,128))
    im = np.array(np.array(im))
    labels[k] = 1
    X[k] = im
    k += 1

X = torch.from_numpy(X).permute(dims=(0,3,1,2))
labels = torch.from_numpy(labels)

## need to fix the shuffle thing. corresponding labels should also shuffle along with X

In [3]:
print(X.max())
X = X/X.max()
mean =  (X[:,0].mean().item(), X[:, 1].mean().item(), X[:,2].mean().item())
std = (X[:,0].std().item(), X[:, 1].std().item(), X[:,2].std().item())
normalize = Normalize(mean, std)
X = normalize(X)
size = 20000
train_X = X[:size]; train_Y = labels[:size]
validation_X = X[size:]; validation_Y = labels[size:]
X = None; labels = None

print(mean, std)

tensor(255.)
(0.4883098900318146, 0.45507100224494934, 0.41694772243499756) (0.25926217436790466, 0.25269168615341187, 0.25525805354118347)


In [4]:
class CustomDataset():
    def __init__(self, X, labels):
        self.data = X
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, indx):
        arr = torch.zeros(2)
        arr[int(self.labels[indx].item())] = 1
        return (self.data[indx], arr)

In [5]:
trainset = CustomDataset(X = train_X, labels = train_Y)
valset = CustomDataset(X = validation_X, labels = validation_Y)

TrainLoader = DataLoader(dataset=trainset, batch_size=20, shuffle=True, num_workers=2)
ValidationLoader = DataLoader(dataset=valset, batch_size=20, shuffle=False, num_workers=2)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Model(nn.Module):
    def __init__(self, lr):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=1) # 6 x 128 x 128
        self.bn1 = nn.BatchNorm2d(num_features=6)
        self.maxpool = nn.MaxPool2d(kernel_size=4) # 6 x 32 x 32

        class Block(nn.Module):
            def __init__(this, features):
                super().__init__()
                this.num_featues = features
                this.conv1 = nn.Conv2d(in_channels=this.num_featues, out_channels=this.num_featues, kernel_size=3, padding=1)
                this.conv2 = nn.Conv2d(in_channels=this.num_featues, out_channels=this.num_featues, kernel_size=3, padding=1)
                this.bn = nn.BatchNorm2d(num_features=this.num_featues)
            def forward(this, x):
                y = torch.relu(this.bn(this.conv1(x)))
                y = this.bn(this.conv2(y))
                return torch.relu(this.bn(x + y))
            
            __call__ = forward

        self.blocks1 = nn.ModuleList([Block(6) for i in range(3)]) # bn1 works
        
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=24, kernel_size=3, padding=1) # 24 x 32 x 32
        self.bn2 = nn.BatchNorm2d(num_features=24)
        # use self.maxpool again -----------------> # 24 x 8 x 8

        self.blocks2 = nn.ModuleList([Block(24) for i in range(3)])

        self.avg_pool = nn.AvgPool2d(kernel_size=2) # 24 x 4 x 4
        self.dropout = nn.Dropout()
        self.bn3 = nn.BatchNorm1d(num_features=32)
        self.l1 = nn.Linear(in_features=24*4*4, out_features=32)
        self.l2 = nn.Linear(in_features=32, out_features=2)

        self.loss = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.parameters(), lr=lr, weight_decay=0.00001)

    def forward(self, x):
        x = self.maxpool(torch.relu(self.bn1(self.conv1(x))))
        for block in self.blocks1:
            x = block(x)
        x = self.maxpool(self.dropout(torch.relu(self.bn2(self.conv2(x)))))
        for block in self.blocks2:
            x = block(x)
        x = self.avg_pool(self.dropout(x))
        x = torch.relu(self.l1(x.view(-1, 24*4*4)))
        x = torch.sigmoid(self.l2(x))
        # x = torch.softmax(self.l2(x), dim=1)

        return x

    def train(self, train_loader: DataLoader, validation_loader: DataLoader|None, epochs = 20):
        epochs += 1
        for epoch in range(epochs):
            for idx, (data, label) in enumerate(train_loader):
                if epoch == 0:
                    continue
                data, label = data.to(device), label.to(device)
                self.optimizer.zero_grad()
                y_pred = self.forward(data)
                if y_pred.isnan().any():
                    print(0)
                    raise Exception("NaN")
                loss = self.loss(label, y_pred)
                loss.backward()
                self.optimizer.step()

            print("Epoch:", epoch)
            with torch.no_grad():
                correct = 0
                total = len(train_loader)*train_loader.batch_size
                for idx, (data, label) in enumerate(train_loader):
                    data, label = data.to(device), label.to(device)
                    y_pred = self.forward(data)
                    correct += (y_pred.argmax(dim=1) == label.argmax(dim=1)).sum().item()

                print(f"Training Data: {correct}/{total}  {correct*100/total}%")
                if validation_loader == None:
                    continue
                correct = 0
                total = len(validation_loader)*validation_loader.batch_size
                for idx, (data, label) in enumerate(validation_loader):
                    data, label = data.to(device), label.to(device)
                    y_pred = self.forward(data)
                    correct += (y_pred.argmax(dim=1) == label.argmax(dim=1)).sum().item()

                print(f"Testing Data: {correct}/{total}  {correct*100/total}%")
            print("===============================================================")


model = Model(lr=0.001)
model.to(device)

model.train(train_loader=TrainLoader, validation_loader=ValidationLoader, epochs=25)

Epoch: 0
Training Data: 9958/20000  49.79%
Testing Data: 2492/5000  49.84%
Epoch: 1
Training Data: 15143/20000  75.715%
Testing Data: 3764/5000  75.28%
Epoch: 2
Training Data: 16117/20000  80.585%
Testing Data: 4020/5000  80.4%
Epoch: 3
Training Data: 16629/20000  83.145%
Testing Data: 4097/5000  81.94%
Epoch: 4
Training Data: 17166/20000  85.83%
Testing Data: 4227/5000  84.54%
Epoch: 5
Training Data: 17313/20000  86.565%
Testing Data: 4240/5000  84.8%
Epoch: 6
Training Data: 17478/20000  87.39%
Testing Data: 4313/5000  86.26%
Epoch: 7
Training Data: 17729/20000  88.645%
Testing Data: 4330/5000  86.6%
Epoch: 8
Training Data: 17947/20000  89.735%
Testing Data: 4350/5000  87.0%
Epoch: 9
Training Data: 17783/20000  88.915%
Testing Data: 4306/5000  86.12%
Epoch: 10
Training Data: 18067/20000  90.335%
Testing Data: 4402/5000  88.04%
Epoch: 11
Training Data: 18014/20000  90.07%
Testing Data: 4360/5000  87.2%
Epoch: 12
Training Data: 18255/20000  91.275%
Testing Data: 4440/5000  88.8%
Epoch: 

In [7]:
model = None

In [8]:
TrainLoader = None
ValidationLoader = None
trainset = None
valset = None

X = torch.concat([train_X, validation_X])
Y = torch.concat([train_Y, validation_Y])

train_X = None; validation_X = None; train_Y = None; validation_Y = None

In [9]:
trainset = CustomDataset(X = X, labels = Y)
TrainLoader = DataLoader(dataset=trainset, batch_size=20, shuffle=True, num_workers=2)

In [10]:
model = Model(lr=0.001)
model.to(device)

model.train(train_loader=TrainLoader, validation_loader=None, epochs=25)

Epoch: 0
Training Data: 12365/25000  49.46%
Epoch: 1
Training Data: 19161/25000  76.644%
Epoch: 2
Training Data: 20152/25000  80.608%
Epoch: 3
Training Data: 20785/25000  83.14%
Epoch: 4
Training Data: 21169/25000  84.676%
Epoch: 5
Training Data: 21414/25000  85.656%
Epoch: 6
Training Data: 21705/25000  86.82%
Epoch: 7
Training Data: 21784/25000  87.136%
Epoch: 8
Training Data: 21974/25000  87.896%
Epoch: 9
Training Data: 21957/25000  87.828%
Epoch: 10
Training Data: 22216/25000  88.864%
Epoch: 11
Training Data: 22376/25000  89.504%
Epoch: 12
Training Data: 22539/25000  90.156%
Epoch: 13
Training Data: 22393/25000  89.572%
Epoch: 14
Training Data: 22622/25000  90.488%
Epoch: 15
Training Data: 22698/25000  90.792%
Epoch: 16
Training Data: 22744/25000  90.976%
Epoch: 17
Training Data: 22913/25000  91.652%
Epoch: 18
Training Data: 22904/25000  91.616%
Epoch: 19
Training Data: 22939/25000  91.756%
Epoch: 20
Training Data: 22949/25000  91.796%
Epoch: 21
Training Data: 23245/25000  92.98%
Ep

In [11]:
torch.save(model.state_dict(),"weights.pth")